In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

pd.options.display.max_rows = 999
pd.options.display.max_columns = None

/usr/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
# Reading and exploring the training dataset
train = pd.read_csv("/home/digant/workspace/pooja/wids2018/train.csv")
train_rows = train.shape[0]
print(train.shape)
train['is_female'].value_counts()

(18255, 1235)


/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (12,49,51,56,91,126,145,163,165,166,167,168,169,171,173,174,176,177,242,244,246,248,252,255,274,290,291,292,294,295,296,332,344,366,374,376,397,414,440,491,620,634,639,642,643,645,710,713,760,769,810,829,929,954,979,1001,1002,1003,1004,1005,1024,1037,1041,1043,1062,1086,1099,1100,1121,1129,1136,1152,1153,1166,1168,1182,1193,1204,1205,1207,1208,1216,1226,1228,1230,1232,1234) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1    9805
0    8450
Name: is_female, dtype: int64

In [13]:
train.dtypes

train_id              int64
AA3                   int64
AA4                   int64
AA5                 float64
AA6                 float64
AA7                   int64
AA14                  int64
AA15                  int64
DG1                   int64
is_female             int64
DG3                   int64
DG3A                  int64
DG3A_OTHERS          object
DG4                   int64
DG4_OTHERS          float64
DG5_1                 int64
DG5_2                 int64
DG5_3                 int64
DG5_4                 int64
DG5_5                 int64
DG5_6                 int64
DG5_7                 int64
DG5_8                 int64
DG5_9                 int64
DG5_10                int64
DG5_11                int64
DG5_96                int64
DG6                   int64
DG8a                  int64
DG8b                  int64
DG8c                  int64
DG9a                float64
DG9b                float64
DG9c                float64
DG10b               float64
DG10c               

In [14]:
# % of missing rows in each column
missin = pd.DataFrame(train.isnull().sum()/train_rows*100)

In [15]:
# Proportion of missing values in each column
missin[missin[0]>50]

,0
AA5,69.033142
DG3A_OTHERS,99.726102
DG4_OTHERS,100.000000
DG9b,54.878116
DG9c,62.728020
DG10b,54.878116
DG10c,62.728020
DG11b,63.971515
DG11c,70.457409
DG12B_1,65.746371


In [4]:
#Reading the test dataset
test = pd.read_csv("/home/digant/workspace/pooja/wids2018/test.csv")
test.shape

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11,13,48,50,55,90,125,144,162,163,164,166,168,169,170,171,172,174,176,241,243,245,247,251,254,273,289,291,292,293,295,296,312,313,314,331,343,365,373,375,396,413,439,490,619,623,630,633,638,642,644,669,712,759,768,788,809,828,928,953,978,1000,1001,1002,1003,1004,1023,1036,1040,1042,1063,1085,1098,1099,1120,1128,1135,1151,1152,1165,1167,1181,1192,1204,1205,1206,1207,1215,1225,1227,1229,1231,1233) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(27285, 1234)

In [5]:
#Create a single dataset with train and test so as to do data preparation only once
df_all = pd.concat([train, test])
df_all.shape

(45540, 1236)

# <font color = 'red'> Taking only the columns present in data dictionary </font>
#### Not all columns were present in the data dictionary. So one assumption was may be the organizers of the competition didnt want participants to use other columns. So this approach considers only the columns present in data dictionary

In [10]:
# read and explore the data dictionary
diction = pd.read_excel("/home/digant/workspace/pooja/wids2018/wids_dd.xls")
print(diction.shape)
diction.head(10)

In [16]:
# Columns present in train but not in data dictionary
b = list(set(train.columns) - set(diction['Column Name']))
print(len(b))
sorted(b)

['AA14',
 'AA15',
 'AA4',
 'AA7',
 'DG10b',
 'DG10c',
 'DG11b',
 'DG11c',
 'DG13_OTHERS',
 'DG14_OTHERS',
 'DG3A_OTHERS',
 'DG4_OTHERS',
 'DL12_OTHERS',
 'DL14',
 'DL1_OTHERS',
 'DL28_OTHERS',
 'DL2_23_OTHERS',
 'DL2_96_OTHERS',
 'DL4_96',
 'DL4_99',
 'DL4_OTHERS',
 'FB16_5_OTHERS',
 'FB16_96_OTHERS',
 'FB19A_OTHERS',
 'FB19B_OTHERS',
 'FB19_6_OTHERS',
 'FB19_96_OTHERS',
 'FB22_7_OTHERS',
 'FB22_96_OTHERS',
 'FB24_OTHERS',
 'FB25_OTHERS',
 'FB26_OTHERS',
 'FB27_OTHERS',
 'FB28_1_OTHERS',
 'FB28_2_OTHERS',
 'FB28_3_OTHERS',
 'FB28_4_OTHERS',
 'FB28_96_OTHERS',
 'FB29_OTHERS',
 'FB4_OTHERS',
 'FF10_OTHERS',
 'FF14_OTHERS',
 'FF2A_OTHERS',
 'FF3_OTHERS',
 'FF7_OTHERS',
 'FL10_OTHERS',
 'FL3_OTHERS',
 'FL4_OTHERS',
 'FL9A_OTHERS',
 'FL9B_OTHERS',
 'G2P1_OTHERS',
 'G2P2_10_OTHERS',
 'G2P2_11_OTHERS',
 'G2P2_12_OTHERS',
 'G2P2_13_OTHERS',
 'G2P2_14_OTHERS',
 'G2P2_15_OTHERS',
 'G2P2_16_OTHERS',
 'G2P2_1_OTHERS',
 'G2P2_2_OTHERS',
 'G2P2_4_OTHERS',
 'G2P2_6_OTHERS',
 'G2P2_7_OTHERS',
 'G2P2_8

In [18]:
# All columns in train present in data dictionary
c = list(set(train.columns)-set(b))
len(c)

In [20]:
# Removing the columns from train not present in data dictionary (asked to be not used)
train1 = train[c]
df_all1 = df_all[c]

### Missing values imputation

In [10]:
# List of all numeric variables
num_var = ['DG8a', 'DG8b', 'DG8c', 'DG9a', 'DG9b', 'DG9c', 'DL8', 'DL11', 'MT1', 'MT6C', 'FF7_1', 'FF7_2', 'FF7_3', 
           'FF7_4', 'FF7_5', 'FF7_6', 'FF7_7', 'FF7_96', 'FF8_1', 'FF8_2', 'FF8_3', 'FF8_4', 'FF8_5', 'FF8_6', 
           'FF8_7', 'FF8_96', 'MM23', 'IFI18', 'FB13', 'FB14', 'FB15', 'DG1','MT11']

In [25]:
#cat_var = list(set(train1.columns) - set(num_var))
cat_var = list(set(df_all1.columns) - set(num_var))

In [26]:
# convert each categorical column to category and then encode each one
# This takes care of missing values as well and encodes each as -1
for col in cat_var:
    #train1[col] = train1[col].astype('category').cat.codes
    df_all1[col] = df_all1[col].astype('category').cat.codes

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [27]:
# Creating a column which describes the number of missing variables for each row
df_all1['tot_nan'] = df_all1.isnull().sum(axis=1)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [29]:
# Replacing NaN values in numerical variables by 0
# Also replacing 99 (dont know) in numerical variables by 0 
for col in num_var:
    #train1.loc[train1[col].isnull(), col] = 0
    #train1.loc[train1[col]==99, col] = 0
    df_all1.loc[df_all1[col].isnull(), col] = 0
    df_all1.loc[df_all1[col]==99, col] = 0

/usr/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [30]:
# Making sure all missing values are taken care of
df_all1.columns[df_all1.isnull().any()]

Index([], dtype='object')

# <font color = 'red'> Taking all columns</font>

In [6]:
#df_all2 = df_all.loc[:,df_all.dtypes != object]
df_all2 = df_all
df_all2.shape

(45540, 1236)

### Feature engineering and missing value imputation

In [7]:
# Since many numeric values are progressive i.e. each _n represnets the nth time of the event, all such can be added to
# get a cumulative variable for all such variables

# All such columns that fall into this logic start with the following start phrases

start_phrase = ['FF14', 'FF19', 'FF6', 'FF7', 'FF8', 'FL6', 'FL7', 'G2P1', 'G2P4', 'G2P5', 'IFI1', 'IFI10', 'IFI11', 
                'IFI12', 'IFI20', 'IFI22', 'IFI23', 'IFI3', 'IFI6', 'IFI7', 'IFI8', 'MM10A', 'MM15', 'MM2', 'MM24', 
                'MM3', 'MM31', 'MM36', 'MM37', 'MM38', 'MM4', 'MM40', 'MM6', 'MMP1', 'MMP3', 'MT16', 'MT18', 'MT4', 
                'RI8']

In [8]:
# add all numeric columns that start with one of the start phrases, but not equal to startphrase

for s in start_phrase:
#     print(s)
    for column in df_all2:
        if df_all2[column].dtype == object:
            continue
            
        if column.startswith(s):
            #print(df_all2[column])
            if s not in df_all2.columns:
                df_all2[s] = 0
            df_all2[s] = df_all2[s] + df_all2[column] # accumulate

In [11]:
#get all numeric variables in addition to num_var which arent present in the data dictionary

ids_all = ['test_id','train_id']
num_var2 = []
num_var_all = []

for column in df_all2:
    if column not in num_var and column not in ids_all:
        if df_all2[column].nunique() > 33: #33 because the variable with most categories as per data dictionary has 32 unique values
            print(column, ' - ',df_all2[column].nunique())
            num_var2.append(column)
#             print(column)

num_var_all = num_var + num_var2
len(num_var_all)

AA14  -  907
AA15  -  451
AA7  -  1050
DL1_OTHERS  -  142
DL2_23_OTHERS  -  96
DL2_96_OTHERS  -  165
DL4_OTHERS  -  208
FB19_6_OTHERS  -  46
FB4_OTHERS  -  49
FF2A_OTHERS  -  50
FL10_OTHERS  -  104
FL9A_OTHERS  -  41
FL9B_OTHERS  -  42
G2P1_OTHERS  -  57
G2P3_1  -  35
G2P3_11  -  38
G2P3_7  -  39
G2P3_8  -  37
G2P3_9  -  45
LN2_RIndLngBEOth  -  73
LN2_WIndLngBEOth  -  83
MM12_REC  -  46
FF6  -  66
MM31  -  49


57

In [12]:
# convert each categorical column to category and then encode each one
# This takes care of missing values as well and encodes each as -1

cat_var_all = list(set(df_all2.columns) - set(num_var_all) - set(ids_all))
len(cat_var_all)

for col in cat_var_all:
    #train1[col] = train1[col].astype('category').cat.codes
    df_all2[col] = df_all2[col].astype('category').cat.codes

In [13]:
# Creating a column which describes the number of missing variables for each row
df_all2['tot_nan'] = df_all2.isnull().sum(axis=1)

In [14]:
df_all2.dtypes.value_counts()

int8       1215
float64      34
object       14
int64        12
dtype: int64

In [15]:
# Remove all the string variables not classified as categorical
df_all3 = df_all2.select_dtypes(exclude=['object'])

In [17]:
df_all3.fillna(0, inplace = True)

/usr/lib/python3.6/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
